In [123]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import holidays
pd.set_option('display.max_rows', 4)

In [124]:
df = pd.read_csv("Vancouver.csv")

In [125]:
# Creating Date table
# Using the same timeframe for both dataset
date_df = df[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']].copy() # Copies columns
date_df = date_df[date_df.YEAR >= 2015]
date_df = date_df.reset_index(drop=True) # Resets the index
date_df.min()
date_df = date_df.rename(columns={"YEAR": "Year", "MONTH": "Month", "DAY": "Day", "HOUR": "Hour", "MINUTE": "Minute"}) # Rename
date_df['Date-key'] = date_df.index # Creates Date_Key column based on index
date_df["Date-key"] += 100 # Adds 100 to every row in the column

In [126]:
date_df.head()

,Year,Month,Day,Hour,Minute,Date-key
0,2019,3,7,2,6,100
1,2019,8,27,4,12,101
...,...,...,...,...,...,...
3,2018,3,2,6,17,103
4,2015,2,4,20,53,104


In [127]:
# Creates a list of datetimes
datetimes = []
for i in range(date_df.shape[0]):
    hour = date_df["Hour"][i]
    if date_df["Hour"][i] > 11:
        date_period = "PM"
    else:
        date_period = "AM"
    month = date_df["Month"][i]
    year = date_df["Year"][i]
    day = date_df["Day"][i]
    minute = date_df["Minute"][i]
    second = "00"
    date_string = str(day) + "-" + str(month) + "-" + str(year) + " " + \
                  str(hour) + ":" + str(minute) + ":" + str(second) + ' ' + date_period
    datetimes.append(datetime.strptime(date_string, '%d-%m-%Y %H:%M:%S %p'))

In [128]:
# Creates the weekend column
weekend = []

for datetime_obj in datetimes:
    if datetime_obj.weekday() < 5:
        weekend.append(False) # is weekday
    else:
        weekend.append(True) # is weekend
date_df["Weekend"] = weekend

In [129]:
# Creates the holidays column
isHoliday = []
ca_holidays = holidays.CA() 
for i in datetimes:
    isHoliday.append(i in ca_holidays)
date_df["Holiday"] = isHoliday

In [130]:
# Creates the holiday name column
holiday_name = []
for i in datetimes:
    holiday_name.append(ca_holidays.get(i))
    
date_df["Holiday-name"] = holiday_name

In [131]:
# Creates the day of week column
weekend = []
day_of_week = []

for date in datetimes:
    day_of_week.append(date.weekday())
date_df["Day-of-week"] = day_of_week

In [132]:
date_df_map = {0: "Monday", 1: "Tuesday", 2: "Wednesday", 
               3: "Thursday", 4: "Friday", 5: "Saturday",
               6: "Sunday"}
date_df.replace({"Day-of-week": date_df_map}, inplace=True)

In [133]:
cols = ["Date-key", "Day-of-week", "Minute","Hour", "Day", "Month"
       , "Year", "Weekend", "Holiday", "Holiday-name"]
date_df = date_df[cols]

In [134]:
date_df

,Date-key,Day-of-week,Minute,Hour,Day,Month,Year,Weekend,Holiday,Holiday-name
0,100,Thursday,6,2,7,3,2019,False,False,None
1,101,Tuesday,12,4,27,8,2019,False,False,None
...,...,...,...,...,...,...,...,...,...,...
193376,193476,Saturday,22,16,4,8,2018,True,False,None
193377,193477,Monday,11,18,20,6,2016,False,False,None


In [135]:
date_df.to_csv('Vancouver_Date_Final.csv', index = False)